<a href="https://colab.research.google.com/github/Sidhtang/CRM-connector-/blob/main/crm_connector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import logging
import json
import requests
import redis
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
import xmlrpc.client  # For Odoo XML-RPC API

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('crm_integration.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Enhanced Data Structures
@dataclass
class UserInput:
    """Stores parsed user input and categorization"""
    raw_input: str
    category: str
    details: Dict[str, Any]
    timestamp: datetime

@dataclass
class IntentContext:
    """Stores parsed user intent and relevant entities with additional metadata"""
    raw_input: str
    intent: str
    entities: Dict[str, Any]
    actions: List[str]
    confidence: float
    timestamp: datetime
    metadata: Dict[str, Any]

@dataclass
class WorkflowResult:
    """Stores the result of workflow execution with detailed metrics"""
    success: bool
    messages: List[str]
    data: Dict[str, Any]
    execution_time: float
    resource_usage: Dict[str, float] = None
    error_details: Optional[Dict[str, Any]] = None

# Simple Secret Manager Implementation
class SimpleSecretManager:
    def __init__(self, config_file: str = "secrets.json"):
        self.config_file = config_file
        try:
            with open(config_file, 'r') as f:
                self.secrets = json.load(f)
        except FileNotFoundError:
            self.secrets = {}
            with open(config_file, 'w') as f:
                json.dump(self.secrets, f)

    def get_secret(self, path: str, key: str) -> str:
        return self.secrets.get(path, {}).get(key)

    def store_secret(self, path: str, key: str, value: str) -> bool:
        try:
            if path not in self.secrets:
                self.secrets[path] = {}
            self.secrets[path][key] = value
            with open(self.config_file, 'w') as f:
                json.dump(self.secrets, f)
            return True
        except Exception as e:
            logging.error(f"Failed to store secret: {str(e)}")
            return False

# Metrics Collection
class MetricsCollector:
    def __init__(self):
        self.redis_client = redis.Redis()

    def record_metric(self, metric_name: str, value: float, tags: Dict[str, str]):
        timestamp = datetime.now().timestamp()
        key = f"metrics:{metric_name}:{timestamp}"
        self.redis_client.hset(key, mapping={
            "value": value,
            "timestamp": timestamp,
            **tags
        })

    def get_metrics(self, metric_name: str, time_range: Tuple[datetime, datetime]) -> pd.DataFrame:
        keys = self.redis_client.keys(f"metrics:{metric_name}:*")
        data = []
        for key in keys:
            metric_data = self.redis_client.hgetall(key)
            if time_range[0].timestamp() <= float(metric_data[b"timestamp"]) <= time_range[1].timestamp():
                data.append(metric_data)
        return pd.DataFrame(data)

# CRM Connector Implementations
class BaseCRMConnector:
    def __init__(self, credentials: Dict[str, str]):
        self.credentials = credentials
        self.metrics = MetricsCollector()

    def authenticate(self) -> bool:
        raise NotImplementedError

    def fetch_data(self, query: Dict[str, Any]) -> List[Dict[str, Any]]:
        raise NotImplementedError

    def push_data(self, data: List[Dict[str, Any]]) -> bool:
        raise NotImplementedError

class OdooConnector(BaseCRMConnector):
    def authenticate(self) -> bool:
        try:
            # Implement Odoo XML-RPC authentication
            self.url = self.credentials.get("url", "http://localhost:8069")
            self.db = self.credentials.get("database", "odoo")
            self.username = self.credentials.get("username", "admin")
            self.password = self.credentials.get("password", "admin")

            # Common endpoint for authentication
            common = xmlrpc.client.ServerProxy(f'{self.url}/xmlrpc/2/common')
            self.uid = common.authenticate(self.db, self.username, self.password, {})

            if self.uid:
                # Initialize models endpoint for data operations
                self.models = xmlrpc.client.ServerProxy(f'{self.url}/xmlrpc/2/object')
                return True
            return False
        except Exception as e:
            logger.error(f"Odoo authentication failed: {str(e)}")
            return False

    def fetch_data(self, query: Dict[str, Any]) -> List[Dict[str, Any]]:
        if not hasattr(self, "uid"):
            raise ValueError("Not authenticated")

        # Implement Odoo search_read operation
        try:
            model = query.get("object_type", "res.partner")
            fields = query.get("fields", ["id", "name", "email", "phone"])
            domain = self._build_odoo_domain(query.get("where", []))
            limit = query.get("limit", 100)

            records = self.models.execute_kw(
                self.db, self.uid, self.password,
                model, 'search_read',
                [domain], {
                    'fields': fields,
                    'limit': limit
                }
            )

            return records
        except Exception as e:
            logger.error(f"Error fetching data from Odoo: {str(e)}")
            raise

    def push_data(self, data: List[Dict[str, Any]]) -> bool:
        if not hasattr(self, "uid"):
            raise ValueError("Not authenticated")

        # Implement data push to Odoo
        try:
            # Determine object type from first record
            if not data:
                return True  # No data to push

            success_count = 0
            for record in data:
                model = record.pop("object_type", "res.partner")

                # Check if record has ID (update) or not (create)
                record_id = record.pop("id", None)

                if record_id:
                    # Update existing record
                    result = self.models.execute_kw(
                        self.db, self.uid, self.password,
                        model, 'write',
                        [[record_id], record]
                    )
                    if result:
                        success_count += 1
                else:
                    # Create new record
                    record_id = self.models.execute_kw(
                        self.db, self.uid, self.password,
                        model, 'create',
                        [record]
                    )
                    if record_id:
                        success_count += 1

            return success_count == len(data)
        except Exception as e:
            logger.error(f"Error pushing data to Odoo: {str(e)}")
            return False

    def _build_odoo_domain(self, filters: List) -> List:
        # Convert query parameters to Odoo domain format
        if not filters:
            return []

        # If filters is already in Odoo domain format, return as is
        if isinstance(filters, list) and all(isinstance(item, tuple) or isinstance(item, list) for item in filters):
            return filters

        # Simple conversion from dict-based filters to Odoo domain
        domain = []
        for field, value in filters.items():
            if isinstance(value, dict):
                operator = value.get('operator', '=')
                val = value.get('value')
                domain.append((field, operator, val))
            else:
                domain.append((field, '=', value))

        return domain

class ZohoCRMConnector(BaseCRMConnector):
    def authenticate(self) -> bool:
        try:
            # Implement Zoho OAuth2 authentication
            self.client_id = self.credentials.get("client_id")
            self.client_secret = self.credentials.get("client_secret")
            self.refresh_token = self.credentials.get("refresh_token")
            self.domain = self.credentials.get("domain", "com")  # Zoho domain (com, eu, in, etc.)

            # Zoho API endpoints
            self.base_url = f"https://www.zohoapis.{self.domain}/crm/v2"
            self.auth_url = f"https://accounts.zoho.{self.domain}/oauth/v2/token"

            # Get access token using refresh token
            auth_data = {
                "refresh_token": self.refresh_token,
                "client_id": self.client_id,
                "client_secret": self.client_secret,
                "grant_type": "refresh_token"
            }

            response = requests.post(self.auth_url, data=auth_data)

            if response.status_code == 200:
                token_data = response.json()
                self.access_token = token_data.get("access_token")
                self.token_expiry = datetime.now().timestamp() + token_data.get("expires_in", 3600)
                self.headers = {
                    "Authorization": f"Zoho-oauthtoken {self.access_token}",
                    "Content-Type": "application/json"
                }
                return True
            else:
                logger.error(f"Zoho authentication failed: {response.text}")
                return False
        except Exception as e:
            logger.error(f"Zoho authentication failed: {str(e)}")
            return False

    def fetch_data(self, query: Dict[str, Any]) -> List[Dict[str, Any]]:
        """
        Fetch data from Zoho CRM using their REST API v2.
        Supports pagination and field selection.
        """
        if not hasattr(self, "access_token"):
            raise ValueError("Not authenticated")

        # Check token expiry and refresh if needed
        self._check_token_expiry()

        try:
            module = query.get("object_type", "Leads")
            fields = query.get("fields", ["id", "Full_Name", "Email", "Phone"])
            where_clause = query.get("where", "")
            limit = query.get("limit", 100)

            # Construct API parameters
            params = {
                "fields": ",".join(fields),
                "per_page": min(limit, 200)  # Zoho's max per page is 200
            }

            # Add search criteria if provided
            if where_clause:
                if isinstance(where_clause, dict):
                    # Convert dict filters to Zoho criteria format
                    criteria = []
                    for field, value in where_clause.items():
                        if isinstance(value, dict):
                            operator = self._map_operator(value.get('operator', '='))
                            val = value.get('value')
                            criteria.append(f"({field}:{operator}:{val})")
                        else:
                            criteria.append(f"({field}:equals:{value})")
                    params["criteria"] = "(" + " and ".join(criteria) + ")"
                else:
                    params["criteria"] = where_clause

            all_records = []
            page = 1

            while True:
                params["page"] = page
                url = f"{self.base_url}/{module}/search"

                response = requests.get(url, headers=self.headers, params=params)

                if response.status_code == 200:
                    data = response.json()
                    records = data.get("data", [])
                    all_records.extend(records)

                    # Check if we have more pages
                    info = data.get("info", {})
                    if info.get("more_records", False) and len(all_records) < limit:
                        page += 1
                    else:
                        break
                else:
                    logger.error(f"Failed to fetch data from Zoho: {response.text}")
                    raise Exception(f"Failed to fetch data: {response.text}")

            return all_records[:limit]

        except Exception as e:
            logger.error(f"Error fetching data from Zoho: {str(e)}")
            raise

    def push_data(self, data: List[Dict[str, Any]]) -> bool:
        """
        Push data to Zoho CRM using their REST API v2.
        Supports both creating new records and updating existing ones.
        """
        if not hasattr(self, "access_token"):
            raise ValueError("Not authenticated")

        # Check token expiry and refresh if needed
        self._check_token_expiry()

        try:
            if not data:
                return True  # No data to push

            success_count = 0
            batch_size = 100  # Zoho allows up to 100 records per request

            # Process records in batches
            for i in range(0, len(data), batch_size):
                batch = data[i:i + batch_size]

                # Separate records for update and insert
                updates = []
                inserts = []

                for record in batch:
                    module = record.pop("object_type", "Leads")
                    record_id = record.pop("id", None)

                    if record_id:
                        record["id"] = record_id
                        updates.append(record)
                    else:
                        inserts.append(record)

                # Handle inserts
                if inserts:
                    url = f"{self.base_url}/{module}"
                    response = requests.post(
                        url,
                        headers=self.headers,
                        json={"data": inserts}
                    )

                    if response.status_code in (200, 201):
                        success_count += len(inserts)
                    else:
                        logger.error(f"Failed to insert records in Zoho: {response.text}")

                # Handle updates
                if updates:
                    url = f"{self.base_url}/{module}"
                    response = requests.put(
                        url,
                        headers=self.headers,
                        json={"data": updates}
                    )

                    if response.status_code == 200:
                        success_count += len(updates)
                    else:
                        logger.error(f"Failed to update records in Zoho: {response.text}")

            return success_count == len(data)

        except Exception as e:
            logger.error(f"Error pushing data to Zoho: {str(e)}")
            return False

    def _check_token_expiry(self):
        """Check if token is expired and refresh if needed"""
        if datetime.now().timestamp() >= self.token_expiry:
            self.authenticate()

    def _map_operator(self, operator: str) -> str:
        """Map common operators to Zoho's operator format"""
        operator_map = {
            '=': 'equals',
            '!=': 'not_equals',
            '>': 'greater_than',
            '<': 'less_than',
            '>=': 'greater_equals',
            '<=': 'less_equals',
            'like': 'contains',
            'not like': 'not_contains',
            'in': 'in',
            'not in': 'not_in',
            'starts with': 'starts_with',
            'ends with': 'ends_with'
        }
        return operator_map.get(operator, 'equals')

# Enhanced Schema Mapping
class SchemaMapper:
    def __init__(self):
        self.metrics = MetricsCollector()

    def map_schema(self, source_data: Dict, source_crm: str, target_crm: str) -> Dict:
        mapping_start = datetime.now()

        # Load schema mapping rules
        mapping_rules = self._load_mapping_rules(source_crm, target_crm)

        # Apply transformations
        mapped_data = self._apply_mapping_rules(source_data, mapping_rules)

        # Validate mapped data
        self._validate_mapped_data(mapped_data, target_crm)

        # Record metrics
        mapping_time = (datetime.now() - mapping_start).total_seconds()
        self.metrics.record_metric(
            "schema_mapping_time",
            mapping_time,
            {
                "source_crm": source_crm,
                "target_crm": target_crm
            }
        )

        return mapped_data

    def _load_mapping_rules(self, source_crm: str, target_crm: str) -> Dict:
        # Load and parse mapping rules from configuration
        with open(f"mapping_rules/{source_crm}_to_{target_crm}.json", "r") as f:
            return json.load(f)

    def _apply_mapping_rules(self, source_data: Dict, mapping_rules: Dict) -> Dict:
        mapped_data = {}
        for target_field, rule in mapping_rules.items():
            if rule["type"] == "direct":
                mapped_data[target_field] = source_data.get(rule["source_field"])
            elif rule["type"] == "transform":
                mapped_data[target_field] = self._apply_transformation(
                    source_data,
                    rule["transformation"]
                )
        return mapped_data

    def _apply_transformation(self, source_data: Dict, transformation_info: Dict) -> Any:
        # Apply transformations based on transformation type
        transform_type = transformation_info.get("type")

        if transform_type == "concat":
            fields = transformation_info.get("fields", [])
            separator = transformation_info.get("separator", " ")
            values = [str(source_data.get(field, "")) for field in fields]
            return separator.join(values)

        elif transform_type == "split":
            field = transformation_info.get("field")
            separator = transformation_info.get("separator", " ")
            index = transformation_info.get("index", 0)
            if field in source_data:
                parts = source_data[field].split(separator)
                if 0 <= index < len(parts):
                    return parts[index]
            return None

        elif transform_type == "date_format":
            field = transformation_info.get("field")
            source_format = transformation_info.get("source_format")
            target_format = transformation_info.get("target_format")
            if field in source_data:
                try:
                    date_obj = datetime.strptime(source_data[field], source_format)
                    return date_obj.strftime(target_format)
                except ValueError:
                    return source_data[field]
            return None

        # Add more transformation types as needed

        return None

    def _validate_mapped_data(self, mapped_data: Dict, target_crm: str):
        # Implement validation logic for target CRM schema
        pass

# Workflow Management with Error Handling
class WorkflowManager:
    def __init__(self):
        self.metrics_collector = MetricsCollector()
        self.input_handler = InputTypeHandler()
        self.schema_mapper = SchemaMapper()

    def process_workflow(self, user_input: str) -> WorkflowResult:
        start_time = datetime.now()
        resource_usage = {
            "memory": 0,
            "cpu": 0,
            "api_calls": 0
        }

        try:
            # Categorize input
            input_data = self.input_handler.categorize_input(user_input)

            # Execute appropriate workflow
            if input_data.category == "data_sync":
                result = self._handle_data_sync(input_data)
                resource_usage["api_calls"] += 2  # Authentication + data fetch
            elif input_data.category == "data_query":
                result = self._handle_data_query(input_data)
                resource_usage["api_calls"] += 1  # Data query
            elif input_data.category == "data_update":
                result = self._handle_data_update(input_data)
                resource_usage["api_calls"] += 1  # Data update
            elif input_data.category == "scheduling":
                result = self._handle_scheduling(input_data)
            elif input_data.category == "reporting":
                result = self._handle_reporting(input_data)
                resource_usage["memory"] += 100  # Report generation
            else:
                raise ValueError(f"Unknown input category: {input_data.category}")

            execution_time = (datetime.now() - start_time).total_seconds()

            return WorkflowResult(
                success=True,
                messages=result.get("messages", []),
                data=result.get("data", {}),
                execution_time=execution_time,
                resource_usage=resource_usage
            )

        except Exception as e:
            logger.error(f"Workflow execution failed: {str(e)}", exc_info=True)
            execution_time = (datetime.now() - start_time).total_seconds()
            return WorkflowResult(
                success=False,
                messages=[f"Error: {str(e)}"],
                data={},
                execution_time=execution_time,
                resource_usage=resource_usage,
                error_details={"error_type": type(e).__name__, "error_message": str(e)}
            )

    def _handle_data_sync(self, input_data: UserInput) -> Dict[str, Any]:
        # Implement data synchronization workflow
        source_system = input_data.details.get("source", "unknown")
        target_system = input_data.details.get("target", "unknown")

        # Set up source and target CRM connectors
        source_connector = self._get_connector(source_system)
        target_connector = self._get_connector(target_system)

        # Authenticate with both systems
        source_authenticated = source_connector.authenticate()
        target_authenticated = target_connector.authenticate()

        if not (source_authenticated and target_authenticated):
            raise Exception(f"Failed to authenticate with {source_system} or {target_system}")

        # Fetch data from source
        source_data = source_connector.fetch_data({"object_type": "Lead", "limit": 100})

        # Map schema between systems
        mapped_data = []
        for record in source_data:
            mapped_record = self.schema_mapper.map_schema(record, source_system, target_system)
            mapped_data.append(mapped_record)

        # Push to target
        success = target_connector.push_data(mapped_data)

        return {
            "messages": [f"Syncing data from {source_system} to {target_system}",
                         f"Synced {len(mapped_data)} records successfully" if success else "Sync failed"],
            "data": {
                "sync_status": "completed" if success else "failed",
                "records_processed": len(source_data),
                "records_synced": len(mapped_data) if success else 0
            }
        }

    def _handle_data_query(self, input_data: UserInput) -> Dict[str, Any]:
        # Implement data query workflow
        query_params = input_data.details.get("query_params", {})
        source_system = input_data.details.get("source", "unknown")

        # Set up connector
        connector = self._get_connector(source_system)

        # Authenticate
        if not connector.authenticate():
            raise Exception(f"Failed to authenticate with {source_system}")

        # Execute query
        results = connector.fetch_data(query_params)

        return {
            "messages": [f"Executed query against {source_system}",
                         f"Found {len(results)} matching records"],
            "data": {"query_results": results}
        }

    def _handle_data_update(self, input_data: UserInput) -> Dict[str, Any]:
        # Implement data update workflow
        target_system = input_data.details.get("target", "unknown")
        records = input_data.details.get("records", [])

        if not records:
            return {
                "messages": ["No records to update"],
                "data": {"update_status": "skipped"}
            }

        # Set up connector
        connector = self._get_connector(target_system)

        # Authenticate
        if not connector.authenticate():
            raise Exception(f"Failed to authenticate with {target_system}")

        # Push updates
        success = connector.push_data(records)

        return {
            "messages": [f"Updating records in {target_system}",
                        f"Updated {len(records)} records successfully" if success else "Update failed"],
            "data": {
                "update_status": "completed" if success else "failed",
                "records_updated": len(records) if success else 0
            }
        }

    def _handle_scheduling(self, input_data: UserInput) -> Dict[str, Any]:
        # Implement scheduling workflow
        schedule_details = input_data.details.get("schedule", {})
        target_system = input_data.details.get("target", "unknown")

        # Validate schedule details
        if not schedule_details.get("task"):
            raise ValueError("No task specified for scheduling")

        # Set up connector for validation
        connector = self._get_connector(target_system)

        # Authenticate to validate target system
        if not connector.authenticate():
            raise Exception(f"Failed to authenticate with {target_system}")

        # Add schedule to database (mock implementation)
        schedule_id = self._create_schedule_entry(schedule_details, target_system)

        return {
            "messages": [f"Scheduled task '{schedule_details.get('task')}' for {target_system}",
                        f"Schedule ID: {schedule_id}"],
            "data": {
                "schedule_id": schedule_id,
                "schedule_details": schedule_details
            }
        }

    def _handle_reporting(self, input_data: UserInput) -> Dict[str, Any]:
        # Implement reporting workflow
        report_type = input_data.details.get("report_type", "summary")
        source_system = input_data.details.get("source", "unknown")
        date_range = input_data.details.get("date_range", {"days": 30})

        # Set up connector
        connector = self._get_connector(source_system)

        # Authenticate
        if not connector.authenticate():
            raise Exception(f"Failed to authenticate with {source_system}")

        # Fetch data for report
        query = self._build_report_query(report_type, date_range)
        data = connector.fetch_data(query)

        # Generate report
        report = self._generate_report(data, report_type)

        return {
            "messages": [f"Generated {report_type} report for {source_system}",
                        f"Report covers data from the last {date_range.get('days')} days"],
            "data": {
                "report": report,
                "metadata": {
                    "report_type": report_type,
                    "date_range": date_range,
                    "record_count": len(data)
                }
            }
        }

    def _get_connector(self, system_name: str) -> BaseCRMConnector:
        """Create and return appropriate connector based on system name"""
        # Get credentials from secret manager
        secret_manager = SimpleSecretManager()
        credentials = {
            "url": secret_manager.get_secret(system_name, "url"),
            "username": secret_manager.get_secret(system_name, "username"),
            "password": secret_manager.get_secret(system_name, "password"),
            "database": secret_manager.get_secret(system_name, "database"),
            "client_id": secret_manager.get_secret(system_name, "client_id"),
            "client_secret": secret_manager.get_secret(system_name, "client_secret"),
            "refresh_token": secret_manager.get_secret(system_name, "refresh_token"),
            "domain": secret_manager.get_secret(system_name, "domain")
        }

        # Create appropriate connector
        if system_name.lower() == "odoo":
            return OdooConnector(credentials)
        elif system_name.lower() == "zoho":
            return ZohoCRMConnector(credentials)
        else:
            raise ValueError(f"Unknown CRM system: {system_name}")

    def _create_schedule_entry(self, schedule_details: Dict, target_system: str) -> str:
        """Create a schedule entry in the database and return schedule ID"""
        # Mock implementation - in real system, this would create an entry in a database
        schedule_id = f"SCH-{datetime.now().strftime('%Y%m%d%H%M%S')}"
        # Here you would save the schedule to a database
        return schedule_id

    def _build_report_query(self, report_type: str, date_range: Dict) -> Dict:
        """Build query parameters based on report type and date range"""
        query = {"limit": 1000}

        # Set appropriate object type based on report type
        if report_type == "leads":
            query["object_type"] = "Lead"
        elif report_type == "opportunities":
            query["object_type"] = "Opportunity"
        elif report_type == "contacts":
            query["object_type"] = "Contact"
        else:
            query["object_type"] = "Lead"  # Default to leads for summary reports

        # Add date range filter
        days = date_range.get("days", 30)
        start_date = (datetime.now() - timedelta(days=days)).strftime("%Y-%m-%d")
        query["where"] = {
            "date_created": {
                "operator": ">=",
                "value": start_date
            }
        }

        # Add fields based on report type
        if report_type == "summary":
            query["fields"] = ["id", "name", "status", "date_created", "assigned_user_id"]
        else:
            query["fields"] = ["id", "name", "status", "date_created", "assigned_user_id",
                              "email", "phone", "source", "description"]

        return query

    def _generate_report(self, data: List[Dict], report_type: str) -> Dict:
        """Generate a structured report from the data"""
        report = {
            "generated_at": datetime.now().isoformat(),
            "total_records": len(data)
        }

        # Generate metrics based on report type
        if report_type == "summary" or report_type == "leads":
            # Group leads by status
            status_counts = {}
            for record in data:
                status = record.get("status", "Unknown")
                status_counts[status] = status_counts.get(status, 0) + 1
            report["status_distribution"] = status_counts

            # Calculate daily new leads
            date_counts = {}
            for record in data:
                date_str = record.get("date_created", "").split("T")[0]
                if date_str:
                    date_counts[date_str] = date_counts.get(date_str, 0) + 1
            report["daily_new_leads"] = date_counts

        elif report_type == "opportunities":
            # Calculate total and average opportunity value
            total_value = 0
            for record in data:
                value = float(record.get("amount", 0))
                total_value += value
            report["total_opportunity_value"] = total_value
            report["average_opportunity_value"] = total_value / len(data) if data else 0

            # Calculate win rate
            won_count = sum(1 for record in data if record.get("status") == "Won")
            report["win_rate"] = won_count / len(data) if data else 0

        # Add raw data for detailed reports
        if report_type != "summary":
            report["records"] = data

        return report

# Main application entry point
def main():
    # Initialize workflow manager
    workflow_manager = WorkflowManager()

    # Example usage
    user_input = "Sync data from Odoo to Zoho"
    logger.info(f"Processing workflow: {user_input}")
    result = workflow_manager.process_workflow(user_input)

    if result.success:
        logger.info("Workflow completed successfully")
        for message in result.messages:
            print(message)
        print(f"Execution time: {result.execution_time:.2f}s")
    else:
        logger.error("Workflow failed")
        for message in result.messages:
            print(message)
        print(f"Error details: {result.error_details}")

if __name__ == "__main__":
    main()